# Evaluation TextGrad with TextualVerifier in Loss

In [1]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker

## Load Dataset

In [2]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


## Setup Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment Iterations

In [4]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [5]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")

        optimizer = TextualGradientDescent([solution])
        
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        # Optimize
        loss_value.backward()
        optimizer.step()

        verification_task_prompts = [
            # Perspective 1: Rule-based verifier (objective, procedural)
            """
            Evaluate the calculation step strictly based on mathematical correctness and procedural rules. 
            If the step violates any algebraic or logical principle, replace it with the corrected version of that step only. 
            Do not proceed to solve the full problem.
            Please proceed output with 
            """,

            # Perspective 2: Teaching assistant (didactic, pedagogical)
            """
            Review the calculation step from the perspective of a teaching assistant helping a student learn. 
            If there's an error or suboptimal explanation, provide a corrected version that would best aid the student's understanding. 
            Focus only on the step in question, without solving the full problem.
            """,

            # Perspective 3: Exam grader (concise, evaluative)
            """
            Assess the calculation step as an exam grader would when evaluating a student's solution. 
            Identify whether the step is mathematically valid and aligns with standard problem-solving conventions. 
            If incorrect, rewrite only the flawed step to reflect what a student should have written to receive full credit. 
            Avoid expanding beyond the current step or solving the entire problem.
            """
        ]

        verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=verification_task_prompts,
            enable_logging=False
        )

        instance = Variable(f"initial_solution: {row_data['raw_solution']}\nloss_value: {original_loss}",
                            requires_grad=False,
                            role_description="instance")

        optimizer_instruction = Variable("""You will optimize $initial_solution based on $loss_value. Be super concise.""",
                                        requires_grad=False,
                                        role_description="optimizer prompt")

        verified_result = verifier.verify(instance=instance,
                                        instruction=optimizer_instruction,
                                        calculation=solution)

        tracker_data = verifier.get_tracker() # Tracker data     

        return {
            "id": row_data["id"],
            "source": row_data["source"],
            "subject": row_data["subject"],
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": "",
            "optimized_solution": solution.value,
            "verified_optimized_solution": verified_result.value,
            "final_solution": verified_result.value,
            "final_solution_answer": extract_answer(verified_result.value), 
            "success": True,
            "error_message": None,
            "processing_time_ms": tracker_data['processing_time_ms'],
            "total_llm_calls": tracker_data['total_llm_calls'],
            "total_input_tokens": tracker_data['total_input_tokens'],
            "total_output_tokens": tracker_data['total_output_tokens'],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "source": "",
            "subject": "",
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "optimized_solution": "",
            "verified_optimized_solution": "",
            "final_solution": "",
            "final_solution_answer": "", 
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [6]:
def run_evaluation():
    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in initial_solution[400:402].iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/result-04-textgrad-tv-o.csv', index=False)
    
    return experiment_df

In [7]:
run_evaluation()

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The energy levels of a hydrogen-like atom are given by\n$$E_n = -\\frac{Z^2}{n^2} E_0$$\nwhere $E_0 = 13.6$ eV is the ionization energy of hydrogen, $Z$ is the atomic number, and $n$ is the principal quantum number.\n\nThe wavelengths of the spectral lines are given by the Rydberg formula:\n$$\\frac{1}{\\lambda} = RZ^2 \\left(\\frac{1}{n_1^2} - \\frac{1}{n_2^2}\\right)$$\nwhere $R$ is the Rydberg constant, $Z$ is the atomic number, and $n_1$ and $n_2$ are the principal quantum numbers of the initial and final states, respectively.\n\nFor hydrogen, $Z=1$, so\n$$\\frac{1}{\\lambda_H} = R \\left(\\frac{1}{n_1^2} - \\frac{1}{n_2^2}\\right)$$\nFor doubly ionized lith

Processing: 100%|██████████| 2/2 [02:54<00:00, 87.17s/it] 

Completed in 174.4 seconds


,id,source,subject,correct_answer,original_solution,original_loss,verified_loss,optimized_solution,verified_optimized_solution,final_solution,final_solution_answer,success,error_message,processing_time_ms,total_llm_calls,total_input_tokens,total_output_tokens,result
0,395,MMLU-CP,college_physics,C,The energy levels of a hydrogen-like atom are ...,The final answer should be $\lambda_{Li} = \f...,,The energy levels of a hydrogen-like atom are ...,<VERIFIED>```\n$$ \frac{1}{\lambda_H} = R \lef...,<VERIFIED>```\n$$ \frac{1}{\lambda_H} = R \lef...,Z,True,None,162593.151855,29,11915,983,{'setup': {'verifier_engine': '<textgrad.engin...
1,406,MMLU-CP,college_physics,B,"The object is thrown horizontally, so the init...",The solution correctly identifies the initial ...,,"The object is thrown horizontally, so the init...",<VERIFIED>```\nΔy = (0 m/s)(2.0 s) + (1/2)(-9....,<VERIFIED>```\nΔy = (0 m/s)(2.0 s) + (1/2)(-9....,Z,True,None,174277.613037,45,23738,948,{'setup': {'verifier_engine': '<textgrad.engin...
